In [1]:
import graph
import random
import patrols
import pickle
import plotter
import stats

g = graph.Graph()
g.loadGraph("./out/saved_graph_races.pkl")
# g.loadGraph("./out/saved_graph.pkl")

real_arrest_counts = {ids:len(nodes.pd_reports) for ids, nodes in g.nodes.items()}
real_arrest_mean = stats.sim_mean(real_arrest_counts)
plotter.imshow_plot(real_arrest_counts, "./out/graphics/LAPD_arrest_count/","All_LA", "LAPD Arrest Record Count",real_arrest_mean, False, False)
# plotter.final_plot(real_arrest_counts, False, "./out/graphics/LAPD_arrest_count/","All_LA", "LAPD Arrest Record Count",real_arrest_mean, False, False)

In [2]:
for pID in range(1,22):
    p_arr_cnt = {ids:len(nodes.pd_reports) for ids, nodes in g.precincts[pID].items()}
    p_arr_mean = stats.sim_mean(p_arr_cnt)
    plotter.imshow_plot(p_arr_cnt, "./out/graphics/LAPD_arrest_count/", f"Prec {pID}", f"LAPD Prec.{pID} Arrest Record Count", p_arr_mean, False, False)
#     plotter.final_plot(p_arr_cnt, True, "./out/graphics/LAPD_arrest_count/", f"Prec {pID}", f"LAPD Prec.{pID} Arrest Record Count", p_arr_mean, False, False)

In [3]:
#run simulations on all of LA
sim_list = []
uniform_arrest_rate = {node:0.35 for node in g.nodes.keys()}

for t in range(10):
    # run 10 trials
    all_nodes = {}

    for pID in range(1,22):
        while(True):
            start = random.choice(list(g.precincts[pID].values()))
            try:
                path, arrests = patrols.random_walk_precinct(45000, g, start, uniform_arrest_rate)
            except:
                continue
            else:
                all_nodes.update(arrests)
                break

    sim_list.append(all_nodes)
    print(f"Trial {t} complete", end="\r")

In [4]:
mystore = stats.result(sim_list)
mean_arr_cnt = {ids: n['mean'] for ids, n in mystore.items()}

ans, (c1,c2,c3) = stats.compare_sims(mystore, real_arrest_counts)

conf_nodes = {ids:sum(tup) for ids, tup in ans.items()}

plotter.imshow_plot(mean_arr_cnt, "./out/graphics/Uniform_Simulation/","All_LA", "Simulated Arrest Count",stats.sim_mean(mean_arr_cnt), False, stats.similarity_score(mystore, real_arrest_counts))


In [5]:
for pID in range(1,22):
    
    real_arrest_counts = {ids:len(nodes.pd_reports) for ids, nodes in g.precincts[pID].items()}

    p_arr_cnt = {ids:mean_arr_cnt[ids] for ids, n in g.precincts[pID].items()}
    p_arr_mean = stats.sim_mean(p_arr_cnt)
    
    plotter.imshow_plot(p_arr_cnt, "./out/graphics/Uniform_Simulation/", f"Prec {pID}", f"Prec.{pID} Simulated Arrest Count", p_arr_mean, False, stats.similarity_score)
#   

In [ ]:
ans, (c85,c95,c99) = stats.compare_sims(mystore, real_arrest_counts)
print(f"nodes with 85 confidence: {c85} \nnodes with 95 confidence: {c95} \nnodes with 99 confidence: {c99}")

print()

print(stats.compare_means(mystore, real_arrest_counts))
for i in range(10):
    print(f"Mean of sim #{i} is {stats.sim_mean(sim_list[i])}")
print(f"Mean of real arrests is {stats.sim_mean(real_arrest_counts)}")

In [ ]:
out_nodes = {node_id:(sum(tup)//3) for node_id, tup in ans.items()}

plotter.basic_plot(out_nodes)

In [ ]:
similarity = stats.similarity_score(mystore, real_arrest_counts)
print(similarity)